In [5]:
##medicine
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:3030/ds")

#"alpha-tocopherol"


def get_med_details(med_name):

    sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX myns: <http://inf558.org/medgraph#>
    select  ?gen_name ?brand_name ?uses ?side_eff ?admin
    where
    {
    ?x a myns:Drugs;
       schema:nonProprietaryName ?gen_name;
    OPTIONAL{?x schema:name ?brand_name}.
    OPTIONAL{?x  myns:Uses ?uses}.
    OPTIONAL{?x  myns:side_effect ?side_eff}.
    OPTIONAL{?x schema:administrationRoute ?admin}.
    FILTER regex(?gen_name,""" + '"' + med_name + '"' + """).
    }""")


    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results
    
#res=set()
#for result in results["results"]["bindings"]:
#    res.add(result["gen_name"]["value"])

In [6]:
get_med_details('alpha-tocopherol')


{'head': {'vars': ['gen_name', 'brand_name', 'uses', 'side_eff', 'admin']},
 'results': {'bindings': [{'gen_name': {'type': 'literal',
     'value': 'alpha-tocopherol'},
    'uses': {'type': 'literal', 'value': 'hemolytic anemia'}},
   {'gen_name': {'type': 'literal', 'value': 'alpha-tocopherol'},
    'uses': {'type': 'literal', 'value': "early-onset Alzheimer's disease"}}]}}

In [7]:
##disease
def get_disease_details(dis_name):

    sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX myns: <http://inf558.org/medgraph#>

    select ?disease_name ?info ?cause ?symptom ?med_name ?specialty_name
    where
    {
    ?x a myns:disease;
       schema:name ?disease_name.

    OPTIONAL{?x  myns:Cause ?cause}.
    OPTIONAL{?x  schema:signOrSymptom ?symptom}.
    OPTIONAL {?x schema:description ?info}.
    OPTIONAL
        {
     ?x  schema:drug ?drug_uri.
    ?drug_uri  schema:nonProprietaryName ?med_name. }
    OPTIONAL
        {
     ?x  schema:relevantSpeciality ?sp_uri.
    ?sp_uri  schema:name ?specialty_name. }

    FILTER (?disease_name=""" + '"' + dis_name + '"' + """)  
     }
    """)


    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results
    

In [45]:
ans=get_disease_details("Crohn's disease")
res=set()
for result in ans["results"]["bindings"]:
    res.add(result["info"]["value"])

In [47]:
###health specialty
def get_specialty_details(specialty_name):

    sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    PREFIX myns: <http://inf558.org/medgraph#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    select  ?specialty_name ?doc_name ?doc_age ?doc_gender ?doc_address ?doc_telephone ?doc_score
    where
    {
    ?x a myns:Specialty;
        schema:name ?specialty_name;
    OPTIONAL{
    ?x	myns:doctors ?doctor_uri.
    OPTIONAL{
    ?doctor_uri schema:name ?doc_name.
    ?doctor_uri  schema:address ?doc_address.
    ?doctor_uri  schema:telephone ?doc_telephone.
    ?doctor_uri  myns:score ?doc_score.
    ?doctor_uri  myns:age ?doc_age.
    ?doctor_uri   schema:gender ?doc_gender.}}
    FILTER (str(?doc_score) != "Not rated yet").
      FILTER (?specialty_name=""" + '"' + specialty_name + '"' + """)
     }
    ORDER BY DESC(?doc_score)
    """)


    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

In [48]:
ans=get_specialty_details('neurology')

In [61]:
res=[]
for result in ans["results"]["bindings"]:
    res.append(result["doc_address"]["value"])